# Tutorial 1: Data preparation for Ghana adaptation example



In [ ]:
#data folder link for MAC
data_folder = "/Users/lena/OneDrive - OnTheHub - The University of Oxford/Work/World Bank/Uruguay/data"
#data folder link for Windows users


In [ ]:
Load libraries

In [ ]:
#Lena to input libraries

Load infrastructure data

In [ ]:
roads = 